In [ ]:
import os
import sys
sys.path.append("..")
from nbr.preparation import Preprocess, save_split, Corpus
from nbr.trainer import NBRTrainer
from nbr.model import RepurchaseModule
import torch
import random
import numpy as np
import optuna
import warnings
warnings.filterwarnings("ignore")

# TaFeng

Fix seed:

In [ ]:
seed = 10
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

Read interactions data (filter users with less than 5 transactions, high purchase frequency and one-day users and items with less than 10 transactions). Train dataset - all baskets except the last two, validation dataset - the last but one basket, test dataset - the last basket:

In [ ]:
corpus_path = "./data/"
dataset_name = "ta_feng"

preprocessor = Preprocess(corpus_path, dataset_name)
preprocessor.load_data(5, 10, filt=True)
save_split(corpus_path, dataset_name, preprocessor)

Before preprocessing: #users = 32266, #items = 23812, #clicks = 817741 (#illegal records = 0)
After preprocessing: #users = 7358, #items = 11202, #clicks = 368951
Saving dataset in ./data//data_ta_feng/...


In [ ]:
corpus = Corpus(corpus_path, dataset_name)
corpus.load_data()

Tune hyperparams on validation dataset:

In [ ]:
trainer = NBRTrainer(
    corpus=corpus,
    max_epochs=3,
    topk=10,
    early_stop_num=None
)

train dataset preparing...


100%|██████████| 7358/7358 [00:12<00:00, 608.62it/s]


dev dataset preparing...


100%|██████████| 7357/7357 [00:01<00:00, 4691.81it/s]


test dataset preparing...


100%|██████████| 7357/7357 [00:01<00:00, 4149.20it/s]


In [ ]:
def objective(trial):
    params = {
        "model": RepurchaseModule(
            item_num=corpus.n_items,
            avg_repeat_interval=corpus.total_avg_interval
        ),
        "batch_size": trial.suggest_categorical("batch_size", [32, 64, 128, 256]),
        "lr": trial.suggest_loguniform("lr", 1e-5, 1e-1),
        "l2_reg_coef": trial.suggest_loguniform("l2_reg_coef", 1e-4, 1e-1)
    }

    trainer.init_hyperparams(**params)
    trainer.train(evaluation_flg=False)
    metrics = trainer.evaluate(mode="dev")
    score = metrics["ndcg"]
    return score

In [ ]:
sampler = optuna.samplers.TPESampler(seed=seed)
study = optuna.create_study(direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=25)

[I 2023-05-02 04:47:18,891] A new study created in memory with name: no-name-c685093d-8d59-4fd3-b0c4-a2a7c6442905


Epoch 1:


Batch loss = 0.647869: 100%|██████████| 8889/8889 [01:12<00:00, 122.22it/s]



Epoch 2:


Batch loss = 0.635774: 100%|██████████| 8889/8889 [01:09<00:00, 127.03it/s]


...


Epoch 2:


Batch loss = 0.659555: 100%|██████████| 8889/8889 [01:17<00:00, 114.76it/s]



Epoch 3:


Batch loss = 0.659433: 100%|██████████| 8889/8889 [01:17<00:00, 114.38it/s]


100%|██████████| 7357/7357 [01:21<00:00, 90.05it/s]
[I 2023-05-02 05:59:29,496] Trial 24 finished with value: 0.10161126104345573 and parameters: {'batch_size': 32, 'lr': 1.0410651356272892e-05, 'l2_reg_coef': 0.00038389638862750913}. Best is trial 14 with value: 0.10280218642963897.


Test Repurchase Module (calculate scores for different seeds):

In [ ]:
trainer = NBRTrainer(
    corpus=corpus,
    max_epochs=20,
    topk=10,
    early_stop_num=3
)

train dataset preparing...


100%|██████████| 7358/7358 [00:13<00:00, 547.96it/s]


dev dataset preparing...


100%|██████████| 7357/7357 [00:01<00:00, 3894.40it/s]


test dataset preparing...


100%|██████████| 7357/7357 [00:02<00:00, 3509.95it/s]


In [ ]:
test_metrics = {
    "precision": [],
    "recall": [],
    "ndcg": []
}

In [ ]:
for seed in range(5):
    print(f"\n___SEED___{seed}")
    torch.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)

    params = {
        "model": RepurchaseModule(
            item_num=corpus.n_items,
            avg_repeat_interval=corpus.total_avg_interval
        ),
        "batch_size": study.best_params["batch_size"],
        "lr": study.best_params["lr"],
        "l2_reg_coef": study.best_params["l2_reg_coef"]
    }

    trainer.init_hyperparams(**params)
    trainer.train()

    metrics = trainer.evaluate(mode="test")

    test_metrics["precision"].append(metrics["precision"])
    test_metrics["recall"].append(metrics["recall"])
    test_metrics["ndcg"].append(metrics["ndcg"])
    print(test_metrics)


___SEED___0
Epoch 1:


Batch loss = 0.676445: 100%|██████████| 1112/1112 [00:18<00:00, 59.75it/s]


Evaluation (dev):



100%|██████████| 7357/7357 [01:25<00:00, 86.03it/s] 


 {'precision': 0.051610710887590054, 'recall': 0.11495819342791812, 'ndcg': 0.10141630054004878}
Epoch 2:


...


Evaluation (dev):



100%|██████████| 7357/7357 [01:19<00:00, 92.05it/s]


 {'precision': 0.05252140818268316, 'recall': 0.11592458742058198, 'ndcg': 0.10216093113759561}



100%|██████████| 7357/7357 [01:18<00:00, 93.88it/s] 

{'precision': [0.05829821938290063, 0.058107924425717, 0.058067146934891935, 0.058175886910425446, 0.05808073943183363], 'recall': [0.12708883872286808, 0.1267917249271807, 0.12742818952983803, 0.12684088059047646, 0.12678311111502852], 'ndcg': [0.11081432366201466, 0.11051284735987134, 0.11087908552800879, 0.11067052948997728, 0.11057756628484917]}


In [ ]:
{
    "precision": np.array(test_metrics["precision"]).mean(),
    "recall": np.array(test_metrics["recall"]).mean(),
    "ndcg": np.array(test_metrics["ndcg"]).mean(),
}

{'precision': 0.05814598341715373,
 'recall': 0.12698654897707834,
 'ndcg': 0.11069087046494426}

# TaoBao

Fix seed:

In [ ]:
seed = 10
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

Read interactions data (filter users with less than 10 transactions, high purchase frequency and one-day users and items with less than 10 transactions). Train dataset - all baskets except the last two, validation dataset - the last but one basket, test dataset - the last basket:

In [ ]:
corpus_path = "./data/"
dataset_name = "taobao"

preprocessor = Preprocess(corpus_path, dataset_name)
preprocessor.load_data(10, 10, filt=True)
save_split(corpus_path, dataset_name, preprocessor)

Before preprocessing: #users = 672404, #items = 638962, #clicks = 2015807 (#illegal records = 0)
After preprocessing: #users = 10092, #items = 22286, #clicks = 67991
Saving dataset in ./data//data_taobao/...


In [ ]:
corpus = Corpus(corpus_path, dataset_name)
corpus.load_data()

Tune hyperparams on validation dataset:

In [ ]:
trainer = NBRTrainer(
    corpus=corpus,
    max_epochs=2,
    topk=10,
    early_stop_num=None
)

train dataset preparing...


100%|██████████| 10092/10092 [00:36<00:00, 274.64it/s]


dev dataset preparing...


100%|██████████| 9307/9307 [00:00<00:00, 35741.19it/s]


test dataset preparing...


100%|██████████| 9307/9307 [00:00<00:00, 30135.90it/s]


In [ ]:
def objective(trial):
    params = {
        "model": RepurchaseModule(
            item_num=corpus.n_items,
            avg_repeat_interval=corpus.total_avg_interval
        ),
        "batch_size": trial.suggest_categorical("batch_size", [32, 64, 128, 256]),
        "lr": trial.suggest_loguniform("lr", 1e-5, 1e-1),
        "l2_reg_coef": trial.suggest_loguniform("l2_reg_coef", 1e-4, 1e-1)
    }

    trainer.init_hyperparams(**params)
    trainer.train(evaluation_flg=False)
    metrics = trainer.evaluate(mode="dev")
    score = metrics["ndcg"]
    return score

In [ ]:
sampler = optuna.samplers.TPESampler(seed=seed)
study = optuna.create_study(direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=25)

[I 2023-05-04 07:47:45,056] A new study created in memory with name: no-name-b27b7ee1-c893-4196-8efc-eee20907a9b3


Epoch 1:


Batch loss = 0.675072: 100%|██████████| 1522/1522 [00:12<00:00, 118.34it/s]


Epoch 2:



Batch loss = 0.674311: 100%|██████████| 1522/1522 [00:12<00:00, 117.82it/s]


...

Epoch 1:


Batch loss = 0.675092: 100%|██████████| 1522/1522 [00:12<00:00, 120.22it/s]



Epoch 2:


Batch loss = 0.674593: 100%|██████████| 1522/1522 [00:12<00:00, 118.20it/s]


100%|██████████| 9307/9307 [02:50<00:00, 54.46it/s]
[I 2023-05-04 09:04:19,830] Trial 24 finished with value: 0.07196471216806259 and parameters: {'batch_size': 32, 'lr': 0.0006484226085149746, 'l2_reg_coef': 0.005136300153259163}. Best is trial 19 with value: 0.07354655117790239.


Test Repurchase Module (calculate scores for different seeds):

In [ ]:
trainer = NBRTrainer(
    corpus=corpus,
    max_epochs=20,
    topk=10,
    early_stop_num=3
)

train dataset preparing...


100%|██████████| 10092/10092 [00:32<00:00, 310.95it/s]


dev dataset preparing...


100%|██████████| 9307/9307 [00:00<00:00, 32143.90it/s]


test dataset preparing...


100%|██████████| 9307/9307 [00:00<00:00, 24237.43it/s]


In [ ]:
test_metrics = {
    "precision": [],
    "recall": [],
    "ndcg": []
}

In [ ]:
for seed in range(3):
    print(f"\n___SEED___{seed}")
    torch.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)

    params = {
        "model": RepurchaseModule(
            item_num=corpus.n_items,
            avg_repeat_interval=corpus.total_avg_interval
        ),
        "batch_size": study.best_params["batch_size"],
        "lr": study.best_params["lr"],
        "l2_reg_coef": study.best_params["l2_reg_coef"]
    }

    trainer.init_hyperparams(**params)
    trainer.train()

    metrics = trainer.evaluate(mode="test")

    test_metrics["precision"].append(metrics["precision"])
    test_metrics["recall"].append(metrics["recall"])
    test_metrics["ndcg"].append(metrics["ndcg"])
    print(test_metrics)


___SEED___0
Epoch 1:


Batch loss = 0.671768: 100%|██████████| 191/191 [00:03<00:00, 53.70it/s]


Evaluation (dev):



100%|██████████| 9307/9307 [02:56<00:00, 52.63it/s]


 {'precision': 0.010454496615450738, 'recall': 0.09900612442247769, 'ndcg': 0.07304581542906191}
Epoch 2:


...


Evaluation (dev):



100%|██████████| 9307/9307 [02:58<00:00, 52.03it/s]


 {'precision': 0.010433007413774578, 'recall': 0.09898821675441424, 'ndcg': 0.07240678834666571}



100%|██████████| 9307/9307 [02:58<00:00, 52.27it/s]

{'precision': [0.011582679703449016, 0.011571935102610939, 0.011550445900934781], 'recall': [0.11188711006052791, 0.11177966405214712, 0.11156477203538555], 'ndcg': [0.08028130504394505, 0.07965814355041348, 0.08039730034099186]}


In [ ]:
{
    "precision": np.array(test_metrics["precision"]).mean(),
    "recall": np.array(test_metrics["recall"]).mean(),
    "ndcg": np.array(test_metrics["ndcg"]).mean(),
}

{'precision': 0.011568353568998246,
 'recall': 0.11174384871602021,
 'ndcg': 0.0801122496451168}

# Dunnhumby

Fix seed:

In [ ]:
seed = 10
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)

Read interactions data (filter users with less than 5 transactions, high purchase frequency and one-day users and items with less than 10 transactions). Train dataset - all baskets except the last two, validation dataset - the last but one basket, test dataset - the last basket:

In [ ]:
corpus_path = "./data/"
dataset_name = "dunnhumby"

preprocessor = Preprocess(corpus_path, dataset_name)
preprocessor.load_data(5, 10, filt=True)
save_split(corpus_path, dataset_name, preprocessor)

Before preprocessing: #users = 2500, #items = 92339, #clicks = 2595370 (#illegal records = 0)
After preprocessing: #users = 2358, #items = 26756, #clicks = 1976796
Saving dataset in ./data//data_dunnhumby/...


In [ ]:
corpus = Corpus(corpus_path, dataset_name)
corpus.load_data()

Tune hyperparams on validation dataset:

In [ ]:
trainer = NBRTrainer(
    corpus=corpus,
    max_epochs=1,
    topk=10,
    early_stop_num=None
)

train dataset preparing...


100%|██████████| 2358/2358 [00:12<00:00, 189.36it/s]


dev dataset preparing...


100%|██████████| 2357/2357 [00:11<00:00, 202.53it/s]


test dataset preparing...


100%|██████████| 2357/2357 [00:10<00:00, 229.09it/s]


In [ ]:
def objective(trial):
    params = {
        "model": RepurchaseModule(
            item_num=corpus.n_items,
            avg_repeat_interval=corpus.total_avg_interval
        ),
        "batch_size": trial.suggest_categorical("batch_size", [32, 64, 128, 256]),
        "lr": trial.suggest_loguniform("lr", 1e-5, 1e-1),
        "l2_reg_coef": trial.suggest_loguniform("l2_reg_coef", 1e-4, 1e-1)
    }

    trainer.init_hyperparams(**params)
    trainer.train(evaluation_flg=False)
    metrics = trainer.evaluate(mode="dev")
    score = metrics["ndcg"]
    return score

In [ ]:
sampler = optuna.samplers.TPESampler(seed=seed)
study = optuna.create_study(direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=25)

[I 2023-05-02 07:42:54,375] A new study created in memory with name: no-name-c4aa80c3-e785-4650-b1b1-69099d384f7b


Epoch 1:


Batch loss = 0.493123: 100%|██████████| 60237/60237 [08:09<00:00, 122.95it/s]


100%|██████████| 2357/2357 [01:14<00:00, 31.48it/s]
[I 2023-05-02 07:52:24,161] Trial 0 finished with value: 0.1571827790007825 and parameters: {'batch_size': 32, 'lr': 0.0009863431872330064, 'l2_reg_coef': 0.0004724870791526793}. Best is trial 0 with value: 0.1571827790007825.


Epoch 1:


...

Batch loss = 0.577396: 100%|██████████| 15060/15060 [02:54<00:00, 86.42it/s]


100%|██████████| 2357/2357 [01:12<00:00, 32.30it/s]
[I 2023-05-02 09:37:07,975] Trial 23 finished with value: 0.16530139755253817 and parameters: {'batch_size': 128, 'lr': 2.0869566476632644e-05, 'l2_reg_coef': 0.0008476179290251597}. Best is trial 23 with value: 0.16530139755253817.


Epoch 1:


Batch loss = 0.577329: 100%|██████████| 15060/15060 [02:53<00:00, 86.90it/s]


100%|██████████| 2357/2357 [01:12<00:00, 32.42it/s]
[I 2023-05-02 09:41:14,003] Trial 24 finished with value: 0.16433026738946085 and parameters: {'batch_size': 128, 'lr': 2.3355591416389358e-05, 'l2_reg_coef': 0.002473103274964016}. Best is trial 23 with value: 0.16530139755253817.


Test Repurchase Module (calculate scores for different seeds):

In [ ]:
trainer = NBRTrainer(
    corpus=corpus,
    max_epochs=20,
    topk=10,
    early_stop_num=3
)

train dataset preparing...


100%|██████████| 2358/2358 [00:09<00:00, 241.97it/s]


dev dataset preparing...


100%|██████████| 2357/2357 [00:11<00:00, 212.88it/s]


test dataset preparing...


100%|██████████| 2357/2357 [00:10<00:00, 218.85it/s]


In [ ]:
test_metrics = {
    "precision": [],
    "recall": [],
    "ndcg": []
}

In [ ]:
for seed in range(3):
    print(f"\n___SEED___{seed}")
    torch.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)

    params = {
        "model": RepurchaseModule(
            item_num=corpus.n_items,
            avg_repeat_interval=corpus.total_avg_interval
        ),
        "batch_size": study.best_params["batch_size"],
        "lr": study.best_params["lr"],
        "l2_reg_coef": study.best_params["l2_reg_coef"]
    }

    trainer.init_hyperparams(**params)
    trainer.train()

    metrics = trainer.evaluate(mode="test")

    test_metrics["precision"].append(metrics["precision"])
    test_metrics["recall"].append(metrics["recall"])
    test_metrics["ndcg"].append(metrics["ndcg"])


___SEED___0
Epoch 1:


Batch loss = 0.577507: 100%|██████████| 15060/15060 [02:46<00:00, 90.62it/s]


Evaluation (dev):



100%|██████████| 2357/2357 [01:10<00:00, 33.57it/s]


 {'precision': 0.11535850657615612, 'recall': 0.17654723061333896, 'ndcg': 0.16435737794573518}
Epoch 2:


...


Batch loss = 0.573862: 100%|██████████| 15060/15060 [02:48<00:00, 89.52it/s]


Evaluation (dev):



100%|██████████| 2357/2357 [01:09<00:00, 33.70it/s]


 {'precision': 0.11591005515485789, 'recall': 0.17607234421162468, 'ndcg': 0.16460485467650643}



100%|██████████| 2357/2357 [01:10<00:00, 33.62it/s]


In [ ]:
{
    "precision": np.array(test_metrics["precision"]).mean(),
    "recall": np.array(test_metrics["recall"]).mean(),
    "ndcg": np.array(test_metrics["ndcg"]).mean(),
}

{'precision': 0.11712629048225144,
 'recall': 0.16806432819513914,
 'ndcg': 0.163914761181492}